# Graph NER Validation Lab

Validação isolada do NER para o retriever de grafo (entidades), com foco em:
- Verificar carregamento correto dos backends (sem fallback regex)
- Construir IDF e avaliar desempenho do slice de grafo
- Comparar backends/parametrizações por dataset

Modelo de embedding de entidade fixo: `sentence-transformers/all-MiniLM-L6-v2`


In [4]:
import sys, os, platform, warnings
from pathlib import Path

# macOS safety
if platform.system() == 'Darwin':
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    print("✓ macOS env set")

import multiprocessing
try:
    multiprocessing.set_start_method('spawn', force=True)
except RuntimeError:
    pass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Optional
from collections import Counter

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# repo root
repo_root = Path.cwd()
while repo_root != repo_root.parent and repo_root.name != 'hybrid-retrieval':
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print(f"Repo root: {repo_root}")


✓ macOS env set
Repo root: /Users/thiago/Documents/GitHub/hybrid-retrieval


In [5]:
# project imports
from src.datasets.loader import load_beir_dataset, select_split, as_documents, as_queries
from src.datasets.schema import Document, Query
from src.retrievers.graph_faiss import GraphRetriever
from src.encoders.entity_encoder import EntityEncoderReal, NERConfig, CacheConfig
from src.encoders.encoders import HFSemanticEncoder
from src.eval.evaluator import evaluate_predictions

print("✓ Imports concluídos")


✓ Imports concluídos


In [6]:
def verify_ner_backend(encoder: EntityEncoderReal, expected_backend: str) -> bool:
    try:
        import spacy
    except Exception:
        print("❌ spaCy não instalado")
        return False
    if encoder._nlp is None:
        print(f"❌ Fallback regex detectado (backend {expected_backend})")
        return False
    ok = isinstance(encoder._nlp, spacy.Language)
    pipes = getattr(encoder._nlp, 'pipe_names', [])
    model = encoder._nlp.meta.get('name', 'unknown') if hasattr(encoder._nlp, 'meta') else 'unknown'
    print(f"✓ NER carregado: {expected_backend} | modelo: {model} | pipes: {pipes}")
    if 'ner' not in pipes:
        print("⚠️ Pipe 'ner' não ativo (a extração pode depender de noun_chunks)")
    return ok

def load_subset_dataset(dataset_name: str, n_docs: int = 150, n_queries: int = 30):
    root = repo_root / 'data' / dataset_name / 'processed' / 'beir'
    corpus, queries, qrels = load_beir_dataset(root)
    split = select_split(qrels, ("test","dev","validation","train"))
    split_eval = "test" if "test" in set(qrels["split"]) else split
    qrels_eval = qrels[qrels["split"] == split_eval].copy()
    qids_with_rel = set(qrels_eval["query_id"].unique())
    queries_eval = queries[queries["query_id"].isin(qids_with_rel)].copy()
    if len(queries_eval) > n_queries:
        queries_eval = queries_eval.head(n_queries)
        qids = set(queries_eval["query_id"].unique())
        qrels_eval = qrels_eval[qrels_eval["query_id"].isin(qids)]
    rel_doc_ids = set(qrels_eval["doc_id"].unique())
    docs_rel = corpus[corpus["doc_id"].isin(rel_doc_ids)].copy()
    if len(docs_rel) < n_docs:
        docs_other = corpus[~corpus["doc_id"].isin(rel_doc_ids)].sample(
            n=min(n_docs - len(docs_rel), len(corpus) - len(docs_rel)), random_state=42
        )
        docs_subset = pd.concat([docs_rel, docs_other]).reset_index(drop=True)
    else:
        docs_subset = docs_rel.head(n_docs)
    docs = as_documents(docs_subset)
    qlist = as_queries(queries_eval)
    print(f"📊 {dataset_name.upper()}: {len(docs)} docs, {len(qlist)} queries, {len(qrels_eval)} qrels")
    return docs, qlist, qrels_eval, [(d.title or '') + ' ' + (d.text or '') for d in docs]


In [8]:
# Carregar datasets (subsets)
print("=== Carregando Datasets ===\n")

datasets = {}
for name in ["scifact", "fiqa", "nfcorpus"]:
    try:
        docs, queries, qrels, doc_texts = load_subset_dataset(name, n_docs=25, n_queries=5)
        datasets[name] = dict(docs=docs, queries=queries, qrels=qrels, doc_texts=doc_texts)
        print(f"✓ {name} carregado\n")
    except Exception as e:
        print(f"⚠️ Erro ao carregar {name}: {e}\n")

print(f"✅ Total: {len(datasets)} datasets carregados")


=== Carregando Datasets ===

📊 SCIFACT: 25 docs, 5 queries, 6 qrels
✓ scifact carregado

📊 FIQA: 25 docs, 5 queries, 11 qrels
✓ fiqa carregado

📊 NFCORPUS: 25 docs, 5 queries, 234 qrels
✓ nfcorpus carregado

✅ Total: 3 datasets carregados


## 1) Baseline NER por domínio (min_df=1, noun_chunks=False, max_entities=256)


In [9]:
GRAPH_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
BASE_PARAMS = dict(min_df=1, ner_use_noun_chunks=False)

# Configurações por dataset
NER_BACKENDS = {
    "scifact": [
        ("scispacy", "en_ner_bionlp13cg_md"),
        ("scispacy", "en_core_sci_md"),
    ],
    "fiqa": [
        ("spacy", "en_core_web_md"),
    ],
    "nfcorpus": [
        ("scispacy", "en_core_sci_md"),
    ],
}

baseline_scores = {}

for ds_name, data in datasets.items():
    print(f"\n--- {ds_name.upper()} ---")
    scores_this = {}
    for backend, model in NER_BACKENDS.get(ds_name, []):
        print(f"  Backend={backend} | model={model}")
        ner_cfg = NERConfig(
            backend=backend,
            model=model,
            use_noun_chunks=BASE_PARAMS["ner_use_noun_chunks"],
            batch_size=16,
            n_process=1,
            allowed_labels=None,
        )
        encoder = EntityEncoderReal(
            graph_model_name=GRAPH_MODEL,
            device="cpu",
            ner=ner_cfg,
            min_df=BASE_PARAMS["min_df"],
            max_entities_per_text=256,
            cache=CacheConfig(artifact_dir=None, force_rebuild=False),
        )
        if not verify_ner_backend(encoder, backend):
            print("  ⚠️ Fallback detectado — pulando")
            continue
        print("  Fit NER+IDF...")
        encoder.fit(data["doc_texts"])  # fit corpus

        # Avaliar retriever
        retriever = GraphRetriever(
            graph_model_name=GRAPH_MODEL,
            device="cpu",
            ner_backend=backend,
            ner_model=model,
            ner_use_noun_chunks=BASE_PARAMS["ner_use_noun_chunks"],
            ner_batch_size=16,
            ner_n_process=1,
            min_df=BASE_PARAMS["min_df"],
            entity_artifact_dir=None,
            entity_force_rebuild=True,
        )
        retriever.build_index(data["docs"])  # encode docs
        results = retriever.retrieve(data["queries"], k=10)
        metrics = evaluate_predictions(results, data["qrels"], ks=(10,))
        nDCG = metrics[metrics["k"]==10]["nDCG"].iloc[0]
        MRR = metrics[metrics["k"]==10]["MRR"].iloc[0]
        scores_this[f"{backend}:{model}"] = dict(nDCG=nDCG, MRR=MRR)
        print(f"  nDCG@10={nDCG:.4f} | MRR@10={MRR:.4f}")
    baseline_scores[ds_name] = scores_this

print("\n=== Baselines NER ===")
print(pd.DataFrame({k: {m: s["nDCG"] for m,s in v.items()} for k,v in baseline_scores.items()}).round(4))



--- SCIFACT ---
  Backend=scispacy | model=en_ner_bionlp13cg_md


/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


✓ NER carregado: scispacy | modelo: ner_bionlp13cg_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF...
2025-11-03 09:41:44 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 09:41:44 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 09:41:44 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 09:41:45 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 722.8ms
2025-11-03 09:41:45 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=269
2025-11-03 09:41:45 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 723.7ms
2025-11-03 09:41:45 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 09:41:47 | INFO     | retriever.graph | [logging.py:220]

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF...
2025-11-03 09:42:13 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 09:42:13 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 09:42:13 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 09:42:14 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 578.3ms
2025-11-03 09:42:14 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 09:42:14 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 579.3ms
2025-11-03 09:42:14 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 09:42:15 | INFO     | retriever.graph | [logging.py:220] | ✓ Enco

## 2) Varredura de parâmetros NER (min_df, noun_chunks, max_entities)

Executa grid pequeno por dataset usando o backend NER vencedor do baseline.


In [10]:
# Seleciona backend vencedor do baseline (hardcoded pelas observações)
BEST_NER = {
    "scifact": ("scispacy", "en_core_sci_md"),
    "fiqa": ("spacy", "en_core_web_md"),
    "nfcorpus": ("scispacy", "en_core_sci_md"),
}

min_df_grid = [1, 2, 3, 5]
noun_chunks_grid = [False, True]
max_entities_grid = [128, 256, 512]

sweep_results = {}

for ds_name, data in datasets.items():
    print(f"\n=== SWEEP: {ds_name.upper()} ===")
    backend, model = BEST_NER[ds_name]
    ds_scores = []
    for min_df in min_df_grid:
        for use_nc in noun_chunks_grid:
            for max_ent in max_entities_grid:
                print(f"  min_df={min_df}, noun_chunks={use_nc}, max_entities={max_ent}")
                ner_cfg = NERConfig(
                    backend=backend,
                    model=model,
                    use_noun_chunks=use_nc,
                    batch_size=16,
                    n_process=1,
                    allowed_labels=None,
                )
                encoder = EntityEncoderReal(
                    graph_model_name=GRAPH_MODEL,
                    device="cpu",
                    ner=ner_cfg,
                    min_df=min_df,
                    max_entities_per_text=max_ent,
                    cache=CacheConfig(artifact_dir=None, force_rebuild=False),
                )
                if not verify_ner_backend(encoder, backend):
                    print("    ⚠️ fallback — pulando")
                    continue
                try:
                    encoder.fit(data["doc_texts"])  # fit
                    retriever = GraphRetriever(
                        graph_model_name=GRAPH_MODEL,
                        device="cpu",
                        ner_backend=backend,
                        ner_model=model,
                        ner_use_noun_chunks=use_nc,
                        ner_batch_size=16,
                        ner_n_process=1,
                        min_df=min_df,
                        entity_artifact_dir=None,
                        entity_force_rebuild=True,
                    )
                    retriever.build_index(data["docs"])  # index
                    results = retriever.retrieve(data["queries"], k=10)
                    metrics = evaluate_predictions(results, data["qrels"], ks=(10,))
                    nDCG = metrics[metrics["k"]==10]["nDCG"].iloc[0]
                    ds_scores.append(dict(min_df=min_df, noun_chunks=use_nc, max_entities=max_ent, nDCG=nDCG))
                    print(f"    nDCG@10={nDCG:.4f}")
                except Exception as e:
                    print(f"    ERRO: {e}")
    sweep_results[ds_name] = pd.DataFrame(ds_scores)

# Exibir melhores por dataset
print("\n=== Melhores parâmetros por dataset ===")
for ds_name, df in sweep_results.items():
    if len(df) == 0:
        print(f"  {ds_name}: sem resultados válidos")
        continue
    best = df.sort_values("nDCG", ascending=False).iloc[0]
    print(f"  {ds_name}: nDCG={best['nDCG']:.4f} com min_df={int(best['min_df'])}, noun_chunks={bool(best['noun_chunks'])}, max_entities={int(best['max_entities'])}")




=== SWEEP: SCIFACT ===
  min_df=1, noun_chunks=False, max_entities=128
✓ NER carregado: scispacy | modelo: core_sci_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
2025-11-03 09:51:18 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 09:51:18 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 09:51:18 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 09:51:19 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 715.4ms
2025-11-03 09:51:19 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=1072
2025-11-03 09:51:19 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 716.2ms
2025-11-03 09:51:19 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 09:51:

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
2025-11-03 09:57:58 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 09:57:58 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 09:57:58 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 09:57:59 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 469.8ms
2025-11-03 09:57:59 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 09:57:59 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 470.5ms
2025-11-03 09:57:59 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 09:58:00 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (G

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
2025-11-03 09:58:05 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 09:58:05 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 09:58:05 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 09:58:05 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 469.8ms
2025-11-03 09:58:05 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 09:58:05 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 470.6ms
2025-11-03 09:58:05 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 09:58:06 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (G

...

✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'parser', 'attribute_ruler', 'ner']
2025-11-03 10:00:40 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:00:40 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:00:40 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:00:40 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 529.2ms
2025-11-03 10:00:40 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=1
2025-11-03 10:00:40 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 530.0ms
2025-11-03 10:00:40 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:00:41 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding docu

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'parser', 'attribute_ruler', 'ner']
2025-11-03 10:00:46 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:00:46 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:00:46 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:00:47 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 530.8ms
2025-11-03 10:00:47 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=1
2025-11-03 10:00:47 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 531.5ms
2025-11-03 10:00:47 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:00:47 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding docu

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'parser', 'attribute_ruler', 'ner']
2025-11-03 10:00:58 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:00:58 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:00:58 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:00:59 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 529.6ms
2025-11-03 10:00:59 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=1
2025-11-03 10:00:59 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 530.4ms
2025-11-03 10:00:59 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:00:59 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding docu

## 3) FIQA: labels financeiras e EntityRuler leve

Testa allowed_labels e padrões leves (ETF, IPO, CDS, CPI, GDP; tickers).


In [11]:
# Executar apenas em FIQA
fiqa = datasets.get("fiqa")
fiqa_scores = []

if fiqa:
    backend, model = ("spacy", "en_core_web_md")
    label_sets = {
        "none": None,
        "finance_core": ["ORG","GPE","MONEY","PERCENT","NORP","PRODUCT"],
    }
    ruler_patterns = [
        {"label":"FIN_TERM","pattern":"ETF"},
        {"label":"FIN_TERM","pattern":"IPO"},
        {"label":"FIN_TERM","pattern":"CDS"},
        {"label":"FIN_TERM","pattern":"CPI"},
        {"label":"FIN_TERM","pattern":"GDP"},
        {"label":"TICKER","pattern":[{"TEXT":{"REGEX":"^[A-Z]{1,5}$"}}]},
    ]
    for labels_name, labels in label_sets.items():
        for use_ruler in [False, True]:
            print(f"  labels={labels_name}, entity_ruler={use_ruler}")
            ner_cfg = NERConfig(
                backend=backend,
                model=model,
                use_noun_chunks=False,
                batch_size=16,
                n_process=1,
                allowed_labels=labels,
                use_entity_ruler=use_ruler,
                ruler_patterns=ruler_patterns if use_ruler else None,
            )
            encoder = EntityEncoderReal(
                graph_model_name=GRAPH_MODEL,
                device="cpu",
                ner=ner_cfg,
                min_df=1,
                max_entities_per_text=256,
                cache=CacheConfig(artifact_dir=None, force_rebuild=True),
            )
            if not verify_ner_backend(encoder, backend):
                print("    ⚠️ fallback — pulando")
                continue
            encoder.fit(fiqa["doc_texts"])  # fit corpus FIQA
            retriever = GraphRetriever(
                graph_model_name=GRAPH_MODEL,
                device="cpu",
                ner_backend=backend,
                ner_model=model,
                ner_use_noun_chunks=False,
                ner_batch_size=16,
                ner_n_process=1,
                min_df=1,
                entity_artifact_dir=None,
                entity_force_rebuild=True,
            )
            retriever.build_index(fiqa["docs"])  # index
            results = retriever.retrieve(fiqa["queries"], k=10)
            metrics = evaluate_predictions(results, fiqa["qrels"], ks=(10,))
            nDCG = metrics[metrics["k"]==10]["nDCG"].iloc[0]
            fiqa_scores.append(dict(labels=labels_name, ruler=use_ruler, nDCG=nDCG))
            print(f"    nDCG@10={nDCG:.4f}")

    df_fiqa = pd.DataFrame(fiqa_scores)
    print("\nFIQA — melhores configs:")
    if len(df_fiqa):
        print(df_fiqa.sort_values("nDCG", ascending=False).head(5).round(4))
    else:
        print("(sem resultados)")
else:
    print("FIQA não carregado")


  labels=none, entity_ruler=False


/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
2025-11-03 10:08:34 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:08:34 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:08:34 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:08:34 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 493.7ms
2025-11-03 10:08:34 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 10:08:34 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 494.5ms
2025-11-03 10:08:34 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:08:35 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (G

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'entity_ruler', 'ner']


...

✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
2025-11-03 10:08:48 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:08:48 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:08:48 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:08:48 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 504.7ms
2025-11-03 10:08:48 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 10:08:48 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 505.5ms
2025-11-03 10:08:48 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:08:49 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (G

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'entity_ruler', 'ner']


/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


2025-11-03 10:08:54 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (25 documentos)
2025-11-03 10:08:54 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 10:08:54 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 10:08:55 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 491.8ms
2025-11-03 10:08:55 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=109
2025-11-03 10:08:55 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 492.6ms
2025-11-03 10:08:55 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 10:08:56 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (Graph) - concluído em 950.2ms
2025-11-03 10:08:56 | INFO     | retriever.graph | [logging.py:

## 4) SciFact e NFCorpus — confirmação rápida do backend

Reporta o melhor backend e os ganhos (a partir dos baselines e/ou sweep).

In [12]:
# Consolidação simples
print("=== Consolidação (melhores) ===")

# Baselines já rodados
try:
    base_table = pd.DataFrame({k: {m: s["nDCG"] for m,s in v.items()} for k,v in baseline_scores.items()}).round(4)
    print("\nBaselines:\n", base_table)
except NameError:
    print("(baselines não estão em memória)")

# Sweep
for ds_name, df in sweep_results.items():
    if len(df) == 0:
        continue
    best = df.sort_values("nDCG", ascending=False).iloc[0]
    print(f"\n{ds_name.upper()} — melhor do sweep: nDCG={best['nDCG']:.4f} | min_df={int(best['min_df'])}, noun_chunks={bool(best['noun_chunks'])}, max_entities={int(best['max_entities'])}")


=== Consolidação (melhores) ===

Baselines:
                                scifact    fiqa  nfcorpus
scispacy:en_ner_bionlp13cg_md   0.5326     NaN       NaN
scispacy:en_core_sci_md         0.7209     NaN    0.1728
spacy:en_core_web_md               NaN  0.6541       NaN

SCIFACT — melhor do sweep: nDCG=0.7209 | min_df=1, noun_chunks=False, max_entities=128

FIQA — melhor do sweep: nDCG=0.6541 | min_df=1, noun_chunks=False, max_entities=128

NFCORPUS — melhor do sweep: nDCG=0.1740 | min_df=2, noun_chunks=False, max_entities=128


## 5) Validação para um subset maior dos datasets

In [18]:
# === Validação com subsets maiores e métricas @5/@10/@20 + Recall@50 ===
from dataclasses import dataclass

# Configs vencedoras por dataset
BEST = {
    "scifact": dict(backend="scispacy", model="en_core_sci_md",  min_df=1, noun_chunks=False, max_entities=128),
    "fiqa":    dict(backend="spacy",    model="en_core_web_md",  min_df=1, noun_chunks=False, max_entities=128),
    "nfcorpus":dict(backend="scispacy", model="en_core_sci_md",  min_df=2, noun_chunks=False, max_entities=128),
}

# Tamanhos maiores (edite se quiser)
BIGGER = {
    "scifact": dict(n_docs=500, n_queries=100),
    "fiqa":    dict(n_docs=500, n_queries=100),
    "nfcorpus":dict(n_docs=1000, n_queries=150),
}

KS = (5, 10, 20)
RECALL_AT = 50

def load_subset(ds_name: str, n_docs: int, n_queries: int):
    # Reusa a função já criada no notebook:
    docs, queries, qrels, doc_texts = load_subset_dataset(ds_name, n_docs=n_docs, n_queries=n_queries)
    return dict(docs=docs, queries=queries, qrels=qrels, doc_texts=doc_texts)

def eval_fixed(ds_name: str, cfg: dict, n_docs: int, n_queries: int, ks=KS, recall_at=RECALL_AT):
    print(f"\n=== {ds_name.upper()} | subset: {n_docs} docs, {n_queries} queries ===")
    data = load_subset(ds_name, n_docs=n_docs, n_queries=n_queries)

    ner_cfg = NERConfig(
        backend=cfg["backend"],
        model=cfg["model"],
        use_noun_chunks=cfg["noun_chunks"],
        batch_size=16,
        n_process=1,
        allowed_labels=None,
    )
    enc = EntityEncoderReal(
        graph_model_name=GRAPH_MODEL,
        device="cpu",
        ner=ner_cfg,
        min_df=cfg["min_df"],
        max_entities_per_text=cfg["max_entities"],
        cache=CacheConfig(artifact_dir=None, force_rebuild=True),
    )
    if not verify_ner_backend(enc, cfg["backend"]):
        print("  ⚠️ Fallback no NER — abortando este dataset.")
        return None

    # Fit IDF das entidades no subset maior
    print("  Fit NER+IDF...")
    enc.fit(data["doc_texts"])

    # Retriever com mesmos params
    retr = GraphRetriever(
        graph_model_name=GRAPH_MODEL,
        device="cpu",
        ner_backend=cfg["backend"],
        ner_model=cfg["model"],
        ner_use_noun_chunks=cfg["noun_chunks"],
        ner_batch_size=16,
        ner_n_process=1,
        min_df=cfg["min_df"],
        entity_artifact_dir=None,
        entity_force_rebuild=True,
    )

    print("  Build index...")
    retr.build_index(data["docs"])

    # Precisamos recuperar até max(RECALL_AT, max(KS))
    topk_needed = max(recall_at, max(ks))
    print(f"  Retrieve top-{topk_needed}...")
    preds = retr.retrieve(data["queries"], k=topk_needed)

    # Métricas em múltiplos @k
    print("  Avaliando métricas...")
    m = evaluate_predictions(preds, data["qrels"], ks=ks)
    # Recall@50
    # Se evaluate_predictions já incluir Recall para k=RECALL_AT, ótimo; se não, recompute:
    if recall_at not in ks:
        m_extra = evaluate_predictions(preds, data["qrels"], ks=(recall_at,))
        recall50 = float(m_extra[m_extra["k"] == recall_at]["Recall"].iloc[0])
    else:
        recall50 = float(m[m["k"] == recall_at]["Recall"].iloc[0])

    # Tabela amigável
    tbl = (m[["k","nDCG","MRR","Recall","Precision"]]
           .sort_values("k")
           .reset_index(drop=True))
    print("\nMétricas por @k:")
    with pd.option_context("display.max_columns", None):
        print(tbl.round(4).to_string(index=False))
    print(f"\nRecall@{recall_at}: {recall50:.4f}")

    return dict(metrics=tbl, recall_at_k=recall50)

# Rodar por dataset com as configs vencedoras
results_bigger = {}
for ds in ["scifact", "fiqa", "nfcorpus"]:
    cfg = BEST[ds]
    sizes = BIGGER[ds]
    res = eval_fixed(ds, cfg, n_docs=sizes["n_docs"], n_queries=sizes["n_queries"], ks=KS, recall_at=RECALL_AT)
    results_bigger[ds] = res

print("\n=== Resumo nDCG@10 e Recall@50 (subset maior) ===")
summary_rows = []
for ds, res in results_bigger.items():
    if not res:
        summary_rows.append(dict(dataset=ds, nDCG10=None, Recall50=None))
        continue
    nDCG10 = float(res["metrics"].loc[res["metrics"]["k"]==10, "nDCG"].iloc[0])
    summary_rows.append(dict(dataset=ds, nDCG10=nDCG10, Recall50=res["recall_at_k"]))
df_summary = pd.DataFrame(summary_rows)
print(df_summary)


=== SCIFACT | subset: 500 docs, 100 queries ===
📊 SCIFACT: 500 docs, 100 queries, 116 qrels
✓ NER carregado: scispacy | modelo: core_sci_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF...
  Build index...
2025-11-03 19:09:38 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (500 documentos)
2025-11-03 19:09:38 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 19:09:38 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 19:09:51 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 12.95s
2025-11-03 19:09:51 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=14047
2025-11-03 19:09:51 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 12.95s
2025-11-03 19:09:51 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Enco

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF...
  Build index...
2025-11-03 19:11:13 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (500 documentos)
2025-11-03 19:11:13 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-11-03 19:11:13 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-11-03 19:11:22 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 9.59s
2025-11-03 19:11:22 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=384, ents=1487
2025-11-03 19:11:22 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 9.59s
2025-11-03 19:11:22 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-11-03 19:11:38 | INFO     | retriever.graph | [logging.p

SciFact: Grafo funciona bem em escala (nDCG@10=0.642, Recall@50=0.90). Conclusão: NER biomédico + parâmetros escolhidos generalizam.

FIQA: Queda forte (nDCG@10=0.131, Recall@50=0.357). Conclusão: com mais dados, a cobertura/precisão de entidades cai; o slice de grafo não sustenta domínio financeiro só com spaCy genérico.

NFCorpus: Muito baixo (nDCG@10=0.093, Recall@50=0.052). Conclusão: desalinhamento de entidade/consulta; grafo isolado tem pouca utilidade aqui.

In [17]:
# === Top-k entidades + Softmax de pesos (por dataset com melhores configs) ===
import faiss
from math import exp
from src.encoders.encoders import l2norm

# Configs vencedoras (ajuste se mudar)
BEST = {
    "scifact": dict(backend="scispacy", model="en_core_sci_md",  min_df=1, noun_chunks=False, max_entities=128),
    "fiqa":    dict(backend="spacy",    model="en_core_web_md",  min_df=1, noun_chunks=False, max_entities=256),  # amplia para mais cobertura
    "nfcorpus":dict(backend="scispacy", model="en_core_sci_md",  min_df=2, noun_chunks=False, max_entities=256),  # amplia para mais cobertura
}

TOPK_LIST = [10, 20, 40]        # top-k entidades por doc/query
USE_SOFTMAX_LIST = [False, True] # comparar sem vs com softmax

def _softmax(weights):
    m = max(weights) if weights else 0.0
    exps = [exp(w - m) for w in weights]
    s = sum(exps) + 1e-12
    return [e / s for e in exps]

class EntityEncoderTopKSoftmax:
    """
    Usa um EntityEncoderReal já 'fit' (ent2idf + _get_emb) e aplica:
      - seleção top-k por score tf*idf
      - pesos: softmax(scores) (ou normalização L1)
    """
    def __init__(self, base_encoder, top_k=20, use_softmax=True):
        self.base = base_encoder
        self.top_k = int(top_k)
        self.use_softmax = bool(use_softmax)
        self.dim = int(self.base.dim)

    def encode_text(self, text: str) -> np.ndarray:
        ents = self.base._extract_entities_batch([text or ""])[0]
        if not ents:
            return np.zeros(self.dim, dtype=np.float32)

        # TF
        tf = {}
        for e in ents:
            if e in self.base.ent2idf:
                tf[e] = tf.get(e, 0) + 1

        if not tf:
            return np.zeros(self.dim, dtype=np.float32)

        # Scores tf*idf
        scored = [(e, tf[e] * self.base.ent2idf[e]) for e in tf.keys()]
        scored.sort(key=lambda x: x[1], reverse=True)
        if self.top_k > 0:
            scored = scored[:self.top_k]

        weights = [s for _, s in scored]
        if self.use_softmax:
            weights = _softmax(weights)
        else:
            s = sum(weights) + 1e-12
            weights = [w / s for w in weights]

        acc = np.zeros(self.dim, dtype=np.float32)
        for (e, _), w in zip(scored, weights):
            acc += float(w) * self.base._get_emb(e)

        return l2norm(acc)

def build_index_and_eval(docs, doc_texts, queries, qrels, wrapper_encoder, ks=(5,10,20), recall_at=50):
    # 1) Encode docs
    doc_vecs = [wrapper_encoder.encode_text(t) for t in doc_texts]
    doc_mat = np.vstack(doc_vecs).astype(np.float32)
    doc_ids = [d.doc_id for d in docs]  # IDs reais que aparecem nos qrels

    # 2) FAISS index
    index = faiss.IndexFlatIP(wrapper_encoder.dim)
    index.add(doc_mat)

    # 3) Retrieve
    topk_needed = max(recall_at, max(ks))
    preds = {}
    for q in queries:
        qv = wrapper_encoder.encode_text(q.text).reshape(1, -1).astype(np.float32)
        scores, idx = index.search(qv, topk_needed)
        ids = [doc_ids[i] for i in idx[0].tolist()]  # usa doc_ids reais
        scs = scores[0].tolist()
        preds[q.query_id] = list(zip(ids, scs))

    # 4) Avaliação
    m = evaluate_predictions(preds, qrels, ks=ks)
    m_extra = evaluate_predictions(preds, qrels, ks=(recall_at,))
    recallX = float(m_extra[m_extra["k"] == recall_at]["Recall"].iloc[0])

    # (opcional) sanity check de overlap
    # rel_ids = set(qrels["doc_id"].unique())
    # pred_ids_any = set(x for lst in preds.values() for x,_ in lst)
    # print("Overlap doc_ids:", len(rel_ids & pred_ids_any))

    return m, recallX

def run_topk_softmax_experiment(ds_name: str, big_n_docs: int, big_n_queries: int):
    print(f"\n=== TOP-K + SOFTMAX | {ds_name.upper()} ===")
    # Carregar subset maior (função já existente no notebook)
    docs, queries, qrels, doc_texts = load_subset_dataset(ds_name, n_docs=big_n_docs, n_queries=big_n_queries)

    cfg = BEST[ds_name]
    # Preparar base encoder
    ner_cfg = NERConfig(
        backend=cfg["backend"],
        model=cfg["model"],
        use_noun_chunks=cfg["noun_chunks"],
        batch_size=16,
        n_process=1,
        allowed_labels=None,
    )
    base_enc = EntityEncoderReal(
        graph_model_name=GRAPH_MODEL,
        device="cpu",
        ner=ner_cfg,
        min_df=cfg["min_df"],
        max_entities_per_text=cfg["max_entities"],
        cache=CacheConfig(artifact_dir=None, force_rebuild=False),
    )
    if not verify_ner_backend(base_enc, cfg["backend"]):
        print("  ⚠️ Fallback no NER — abortando.")
        return None

    print("  Fit NER+IDF (base)...")
    base_enc.fit(doc_texts)

    rows = []
    for top_k in TOPK_LIST:
        for use_softmax in USE_SOFTMAX_LIST:
            print(f"  >> top_k={top_k}, softmax={use_softmax}")
            wrap = EntityEncoderTopKSoftmax(base_enc, top_k=top_k, use_softmax=use_softmax)
            # dentro do loop top_k/use_softmax:
            m, recall50 = build_index_and_eval(
                docs=docs,
                doc_texts=doc_texts,
                queries=queries,
                qrels=qrels,
                wrapper_encoder=wrap,
                ks=(5,10,20),
                recall_at=50
            )
            row = dict(top_k=top_k, softmax=use_softmax,
                       nDCG5=float(m[m['k']==5]['nDCG'].iloc[0]),
                       nDCG10=float(m[m['k']==10]['nDCG'].iloc[0]),
                       nDCG20=float(m[m['k']==20]['nDCG'].iloc[0]),
                       Recall50=recall50)
            rows.append(row)
            print(f"     nDCG@10={row['nDCG10']:.4f} | Recall@50={row['Recall50']:.4f}")

    df = pd.DataFrame(rows).sort_values(["nDCG10","Recall50"], ascending=False)
    print("\nResultados (ordenado por nDCG@10):")
    with pd.option_context("display.max_columns", None):
        print(df.round(4).to_string(index=False))
    return df

# Tamanhos maiores (ajuste se quiser)
BIG = {"scifact": (800, 150), "fiqa": (800, 150), "nfcorpus": (1500, 200)}

dfs_topk = {}
for ds in ["scifact", "fiqa", "nfcorpus"]:
    n_docs, n_queries = BIG[ds]
    dfs_topk[ds] = run_topk_softmax_experiment(ds, big_n_docs=n_docs, big_n_queries=n_queries)


=== TOP-K + SOFTMAX | SCIFACT ===
📊 SCIFACT: 800 docs, 150 queries, 171 qrels
✓ NER carregado: scispacy | modelo: core_sci_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF (base)...
  >> top_k=10, softmax=False
     nDCG@10=0.5138 | Recall@50=0.7984
  >> top_k=10, softmax=True
     nDCG@10=0.3142 | Recall@50=0.5443
  >> top_k=20, softmax=False
     nDCG@10=0.5375 | Recall@50=0.8451
  >> top_k=20, softmax=True
     nDCG@10=0.3142 | Recall@50=0.5443
  >> top_k=40, softmax=False
     nDCG@10=0.5458 | Recall@50=0.8803
  >> top_k=40, softmax=True
     nDCG@10=0.3142 | Recall@50=0.5443

Resultados (ordenado por nDCG@10):
 top_k  softmax  nDCG5  nDCG10  nDCG20  Recall50
    40    False 0.5150  0.5458  0.5724    0.8803
    20    False 0.5075  0.5375  0.5668    0.8451
    10    False 0.4853  0.5138  0.5329    0.7984
    10     True 0.3054  0.3142  0.3342    0.5443
    20     True 0.3054  0.3142  0.3342    0.5443
    40     True 0.3054  0.3142  0.3342    0.5443

=== TOP-K + SOFTM

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✓ NER carregado: spacy | modelo: core_web_md | pipes: ['tok2vec', 'attribute_ruler', 'ner']
  Fit NER+IDF (base)...
  >> top_k=10, softmax=False
     nDCG@10=0.1120 | Recall@50=0.2574
  >> top_k=10, softmax=True
     nDCG@10=0.1059 | Recall@50=0.2272
  >> top_k=20, softmax=False
     nDCG@10=0.1126 | Recall@50=0.2591
  >> top_k=20, softmax=True
     nDCG@10=0.1058 | Recall@50=0.2272
  >> top_k=40, softmax=False
     nDCG@10=0.1125 | Recall@50=0.2607
  >> top_k=40, softmax=True
     nDCG@10=0.1058 | Recall@50=0.2272

Resultados (ordenado por nDCG@10):
 top_k  softmax  nDCG5  nDCG10  nDCG20  Recall50
    20    False 0.1030  0.1126  0.1244    0.2591
    40    False 0.1030  0.1125  0.1250    0.2607
    10    False 0.1025  0.1120  0.1226    0.2574
    10     True 0.1012  0.1059  0.1147    0.2272
    20     True 0.1002  0.1058  0.1146    0.2272
    40     True 0.1002  0.1058  0.1146    0.2272

=== TOP-K + SOFTMAX | NFCORPUS ===
📊 NFCORPUS: 1500 docs, 200 queries, 9488 qrels
✓ NER carregado: 